<a href="https://colab.research.google.com/github/mwestt/BMI707-Project/blob/master/Playground.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Playground

Use this notebook to import functions and play around with specific model architectures. If you're running this in colab, we'll need to clone the repo this notebook is in, as well as the repo with the data in. 

**Make sure you comment out this next cell if running locally!**

In [2]:
# Clone our project repo so we've got our code
!git clone https://github.com/mwestt/BMI707-Project.git

# Clone covid-chestxray-dataset repo for data and metadata
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

# Move data and metadata to project repo and cd to it
!mv covid-chestxray-dataset/images/ BMI707-Project/
!mv covid-chestxray-dataset/metadata.csv BMI707-Project/
%cd BMI707-Project/

fatal: destination path 'BMI707-Project' already exists and is not an empty directory.
fatal: destination path 'covid-chestxray-dataset' already exists and is not an empty directory.
mv: cannot stat 'covid-chestxray-dataset/images/': No such file or directory
mv: cannot stat 'covid-chestxray-dataset/metadata.csv': No such file or directory
/content/BMI707-Project


## Benchmark Classifier - No Augmentation

First, we'll need to import the necessary functions from the codebase

In [3]:
from load_data import load_metadata, load_data
from benchmark_classifier import train_benchmark

# Load the metadata csv
df_train, df_val = load_metadata('metadata.csv')
df_val.head()

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
9,3,4.0,M,74.0,SARS,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,2004,"Mount Sinai Hospital, Toronto, Ontario, Canada",images,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0....,10.1148/rg.242035193,https://pubs.rsna.org/doi/10.1148/rg.242035193,NaN,SARS in a 74-year-old man who developed sympto...,NaN,NaN
316,178,1.0,F,72.0,COVID-19,N,Y,Y,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,PA,X-ray,NaN,"Hospital Universitario Doctor Peset, Valencia,...",images,16660_3_1.jpg,NaN,https://www.eurorad.org/case/16660,CC BY-NC-SA 4.0,A 72-year-old woman admitted with acute respir...,NaN,NaN
183,96,0.0,M,60.0,"COVID-19, ARDS",NaN,NaN,NaN,Y,NaN,NaN,NaN,89.0,NaN,NaN,NaN,PA,X-ray,2020,Spain,images,covid-19-pneumonia-rapidly-progressive-admissi...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,Fever and odynophagia. Trip to Italy 7 days ag...,"Case courtesy of Dr Edgar Lorente, Radiopaedia...",NaN
273,154,10.0,NaN,NaN,COVID-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AP,X-ray,2020,NaN,images,radiol.2020201160.fig2d.jpeg,10.1148/radiol.2020201160,https://pubs.rsna.org/doi/full/10.1148/radiol....,NaN,NaN,NaN,NaN
101,51,3.0,M,47.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,39.0,95.0,NaN,NaN,NaN,PA,X-ray,"March 4, 2020",Italy,images,F4341CE7-73C9-45C6-99C8-8567A5484B63.jpeg,NaN,https://www.sirm.org/2020/03/10/covid-19-caso-34/,NaN,"Male patient, 47 years old. Remote history cha...","Credit to G.Patelli , F.Besana , S. Paganoni *...",NaN


In [0]:
# Load training and validation images from metadata csv's
images_train, labels_train = load_data(df_train, image_size=(256, 256))
images_val, labels_val = load_data(df_val, image_size=(256, 256))

Here we'll define the Conv Net with the function `playground_model()`. Toy around with some of the parameters and see if you can get a decent-looking AUC (I'd say we're aiming for **0.7** but maybe we can do better). In general we want as simple a model as possible to be able to get perfect training accuracy, and see how good we can get our validation AUC. Some things to try:

Mainly: 
- **Add extra or remove existing `Conv2D` layers.**
- **Change number of filters in each `Conv2D` layer (first argument)**

*But also:*
- Add or remove `Dropout` layers (these layers are probably unnecessary here)
- Change max pooling layers to average pooling
- Smaller `Dense` layer in the final layer
- Global average pooling instead of the final `Dense` layer

**Make sure you're using a GPU Runtime!** Go to *Runtime > Change runtime type > Hardware accelerator > GPU*

In [9]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from sklearn.metrics import roc_auc_score


def playground_model(input_shape=(256, 256)):
    """Create Keras model using Sequential API.
    
    Returns
    -------
    model : Keras Sequential object
        Keras Sequential model following the specified architecture.
    """

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(input_shape[0], input_shape[1], 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    # model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    return model

# Assign the model defined above
model = playground_model()

# Train benchmark model - if you find good parameters, set `save=True`
trained_model = train_benchmark(model, images_train, images_val, labels_train, labels_val, 
                                epochs=12, batch_size=12, save=True)

# Evaluation metrics - pay attention to AUC!
print('Validation Labels:')
print(labels_val)

print('Predicted Labels:')
y_pred = trained_model.predict_classes(images_val, verbose=1).T[0]
print(y_pred)

print('Predicted Probabilities')
y_probs = trained_model.predict(images_val, verbose=1).T[0]
print(y_probs)

print('Prediction AUC')
print(roc_auc_score(labels_val, y_probs))

Train on 189 samples, validate on 94 samples
Epoch 1/12
189/189 [==============================] - 2s 8ms/step - loss: 7.6271 - accuracy: 0.7037 - val_loss: 0.5660 - val_accuracy: 0.8298
Epoch 2/12
189/189 [==============================] - 1s 8ms/step - loss: 0.5165 - accuracy: 0.7884 - val_loss: 0.5277 - val_accuracy: 0.8191
Epoch 3/12
189/189 [==============================] - 1s 8ms/step - loss: 0.4702 - accuracy: 0.8307 - val_loss: 0.5062 - val_accuracy: 0.8191
Epoch 4/12
189/189 [==============================] - 1s 8ms/step - loss: 0.3182 - accuracy: 0.8730 - val_loss: 0.5478 - val_accuracy: 0.8191
Epoch 5/12
189/189 [==============================] - 1s 8ms/step - loss: 0.2137 - accuracy: 0.9153 - val_loss: 0.6030 - val_accuracy: 0.7128
Epoch 6/12
189/189 [==============================] - 1s 8ms/step - loss: 0.1983 - accuracy: 0.9259 - val_loss: 0.6171 - val_accuracy: 0.7340
Epoch 7/12
189/189 [==============================] - 1s 8ms/step - loss: 0.1845 - accuracy: 0.9524 - v

## Loading Model Workflow

Feel free to add any code cells following this, for example loading your saved models and then using them to make predictions. For example, here's one I made earlier! Just save your own model with `save=True` in `train_benchmark()` above,
and replace the filepath below with your own.

In [10]:
# Load saved model and print summary information
loaded_model = load_model('model_bench_2020_04_26_00_37_24.h5')    
print(loaded_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 256)     73984     
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 256)     0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1048576)          

In [12]:
print('Validation Labels:')
print(labels_val)

print('Predicted Labels:')
y_pred = loaded_model.predict_classes(images_val, verbose=1).T[0]
print(y_pred)

print('Predicted Probabilities')
y_probs = loaded_model.predict(images_val, verbose=1).T[0]
# print(y_probs)

print('Prediction AUC')
print(roc_auc_score(labels_val, y_probs))

print('Prediction Accuracy')
print(sum(y_pred == labels_val) / len(labels_val))

Validation Labels:
[0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
Predicted Labels:
94/94 [==============================] - 0s 2ms/step
[1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1
 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0
 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
Predicted Probabilities
94/94 [==============================] - 0s 2ms/step
Prediction AUC
0.672275641025641
Prediction Accuracy
0.7340425531914894


# Training with Data Augmentation

The following cell is much like the previous section, but we will now train on augmented data using Keras data augmentation, rather than on the images directly. There are a number of augmentation parameters to explore, take a look at the Keras documentation for the [`ImageDataGenerator` class](https://keras.io/preprocessing/image/) for potential arguments to try.

In [13]:
from load_data import data_generator_from_dataframe, ValidImageDataGenerator
from benchmark_classifier import train_augmented_benchmark
from keras.preprocessing.image import ImageDataGenerator

image_size = (256, 256)
batch_size = 8
epochs = 12

# Effective size of training set
n_train_eff = 200
n_val_eff = 60
steps_per_epoch = n_train_eff // batch_size
validation_steps = n_val_eff // batch_size

# Define ImageDataGenerator for training - tweak these arguments
train_datagen = ImageDataGenerator(rescale=1./255
                                   )

train_generator = data_generator_from_dataframe(train_datagen, df_train,
                                                image_size=image_size, 
                                                batch_size=batch_size)

# Simple generator for validation
validation_generator = data_generator_from_dataframe(ValidImageDataGenerator(), 
                                                     df_val, image_size=image_size, 
                                                     batch_size=batch_size)

# Reinstantiate model
model = playground_model(input_shape=image_size)

# Train model with augmented data
trained_model_aug = train_augmented_benchmark(model, train_generator, 
                                              validation_generator,
                                              epochs=epochs, 
                                              steps_per_epoch=steps_per_epoch, 
                                              validation_steps=validation_steps, 
                                              save=False)

# Evaluation metrics - once again, pay attention to AUC!
print('Validation Labels:')
print(labels_val)

# images_val, labels_val = load_data(df_val, image_size=image_size)
print('Predicted Labels:')
y_pred = trained_model_aug.predict_classes(images_val, verbose=1).T[0]
print(y_pred)

print('Predicted Probabilities')  
y_probs = trained_model_aug.predict(images_val, verbose=1).T[0]
print(y_probs)

print('Prediction AUC')
print(roc_auc_score(labels_val, y_probs))

print('Prediction Accuracy')
print(sum(y_pred == labels_val) / len(labels_val))

Found 189 validated image filenames.
Found 94 validated image filenames.
Epoch 1/12
25/25 [==============================] - 10s 413ms/step - loss: 1.9868 - accuracy: 0.7423 - val_loss: 0.1333 - val_accuracy: 0.8036
Epoch 2/12
25/25 [==============================] - 9s 344ms/step - loss: 0.4871 - accuracy: 0.7900 - val_loss: 0.1729 - val_accuracy: 0.8889
Epoch 3/12
25/25 [==============================] - 9s 371ms/step - loss: 0.3018 - accuracy: 0.8579 - val_loss: 1.3126 - val_accuracy: 0.7857
Epoch 4/12
25/25 [==============================] - 10s 402ms/step - loss: 0.2005 - accuracy: 0.9239 - val_loss: 0.8198 - val_accuracy: 0.7037
Epoch 5/12
25/25 [==============================] - 7s 286ms/step - loss: 0.1028 - accuracy: 0.9588 - val_loss: 0.6131 - val_accuracy: 0.6964
Epoch 6/12
25/25 [==============================] - 12s 468ms/step - loss: 0.1478 - accuracy: 0.9450 - val_loss: 0.5267 - val_accuracy: 0.6481
Epoch 7/12
25/25 [==============================] - 7s 265ms/step - loss